In [1]:
import datetime
import gc
import itertools
import json
import logging
import os
import shutil
import math
from pathlib import Path

import albumentations as A
import cv2
import japanize_matplotlib
import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F

from albumentations import ImageOnlyTransform
from albumentations.pytorch import ToTensorV2
from cosine_annealing_warmup import CosineAnnealingWarmupRestarts
from PIL import Image
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from torch.cuda.amp import GradScaler, autocast
from torch.optim import Adam, AdamW
from torch.optim.lr_scheduler import (
    CosineAnnealingLR,
    CosineAnnealingWarmRestarts,
    MultiStepLR,
    ReduceLROnPlateau,
)
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from transformers import get_cosine_schedule_with_warmup, get_linear_schedule_with_warmup
from torch.utils.tensorboard import SummaryWriter
import faiss
from pytorch_metric_learning import losses

from src.const import OUTPUT_ROOT, DATA_ROOT, QUERY_IMAGES_PATH, CITE_IMAGES_PATH

In [3]:
class Config:
    competition = "sake"
    name = "meigara_classification_with_arcface"

    debug = False

    training = True
    evaluation = True
    embedding = True

    seed = 8823
    n_fold = 5
    trn_fold = [0,1,2,3,4]

    target_columns = ["meigara_label"]
    size = 512

    model_name = "tf_efficientnet_b0_ns"
    max_epochs = 5
    train_batch_size = 16
    valid_batch_size = 64
    num_workers = 4
    gradient_accumulation_steps = 1
    clip_grad_norm = 1000

    optimizer = dict(
        optimizer_name="AdamW",
        lr=1e-3,
        weight_decay=1e-2,
        eps=1e-6,
        beta=(0.9, 0.999),
        encoder_lr=1e-4,
        decoder_lr=1e-4,
    )

    scheduler = dict(
        scheduler_name="cosine",
        num_warmup_steps_rate=0,
        num_cycles=0.5,
    )
    batch_scheduler = True


if Config.debug:
    Config.max_epochs = 2
    Config.n_fold = 2
    Config.trn_fold = [0, 1]
    Config.name = Config.name + "_debug"
    Config.size = 128
    Config.train_batch_size = 8

In [4]:
# constants
HOME = OUTPUT_ROOT / 'train_arcface'
EXP_NAME = Config.name
INPUTS = DATA_ROOT  # input data
OUTPUTS = HOME / "outputs"
INTERMIDIATES = HOME / "intermidiates"  # intermidiate outputs
SUBMISSIONS = HOME / "submissions"
OUTPUTS_EXP = OUTPUTS / EXP_NAME
EXP_MODELS = OUTPUTS_EXP / "models"
EXP_REPORTS = OUTPUTS_EXP / "reports"
EXP_PREDS = OUTPUTS_EXP / "predictions"

CITE_IMAGES = CITE_IMAGES_PATH
QUERY_IMAGES = QUERY_IMAGES_PATH

In [19]:
class SimilaritySearcher:
    def __init__(self, embeddings):
        self.dimension = embeddings.shape[1]
        self.index = faiss.IndexFlatL2(self.dimension)
        faiss.normalize_L2(embeddings)
        self.index.add(embeddings)  # type:ignore

    def search(self, queries, k=10):
        assert queries.shape[1] == self.dimension, "Query dimensions should match embeddings dimension."
        faiss.normalize_L2(queries)
        D, I = self.index.search(queries, k)  # type:ignore
        return D, I

In [29]:
cite_features = joblib.load(EXP_PREDS / "cite_embeddings.pkl")["embeddings_list"]
query_features = joblib.load(EXP_PREDS / "test_embeddings.pkl")["embeddings_list"]

ave_cite_feature = np.hstack(cite_features)
ave_query_feature = np.hstack(query_features)

searcher = SimilaritySearcher(ave_cite_feature.astype(np.float32))
D, I = searcher.search(ave_query_feature.astype(np.float32), k=20)

In [31]:
ave_cite_feature.shape

(109481, 2560)

In [17]:
def make_train_label(train_df) -> pd.DataFrame:
    le = LabelEncoder()
    train_df["brand_id_label"] = le.fit_transform(train_df["brand_id"])
    train_df["meigara_label"] = le.fit_transform(train_df["meigara"])

    return train_df


def make_filepath(input_df):
    def join_path(dirpath, filename):
        return (dirpath / filename).as_posix()

    output_df = input_df.assign(
        filepath=input_df["filename"].apply(
            lambda x: join_path(QUERY_IMAGES, x) if str(x)[0] == "2" else join_path(CITE_IMAGES, x)
        )
    )
    return output_df

def make_label_dict(train_df):
    MLABEL2MEIGARA = train_df[["meigara", "meigara_label"]].set_index("meigara_label").to_dict()["meigara"]
    BLABEL2BLAND = train_df[["brand_id", "brand_id_label"]].set_index("brand_id_label").to_dict()["brand_id"]
    return MLABEL2MEIGARA, BLABEL2BLAND

In [18]:
# load raw data
train_df = pd.read_csv(INPUTS / "train.csv")
test_df = pd.read_csv(INPUTS / "test.csv")
cite_df = pd.read_csv(INPUTS / "cite.csv").rename(columns={"cite_gid":"gid", "cite_filename":"filename"})
sample_submission_df = pd.read_csv(INPUTS / "sample_submission.csv")

if Config.debug:
    train_df = train_df.sample(1000, random_state=Config.seed).reset_index(drop=True)
    cite_df = cite_df.sample(100, random_state=Config.seed).reset_index(drop=True)

# make label
train_df = make_train_label(train_df)
MLABEL2MEIGARA, BLABEL2BLAND = make_label_dict(train_df)
TARGET_DIM = 512  # arcfaceの埋め込み次元

# make filepath
train_df = make_filepath(train_df)
test_df = make_filepath(test_df)
cite_df = make_filepath(cite_df)

# for submission
IDX2CITE_GID = cite_df.to_dict()["gid"]

print(train_df.shape)

(108160, 7)


In [13]:
def make_submission(indices):
    vfunc = np.vectorize(lambda x: IDX2CITE_GID[x])
    gid_array = vfunc(I)
    submission_df = test_df[["gid"]].assign(cite_gid=[" ".join(list(x)) for x in gid_array.astype(str)])
    return submission_df

In [33]:
submission_df = make_submission(indices=I)
submission_df.to_csv(SUBMISSIONS / f"{Config.name}_hstack.csv", index=False)